#### 1. Collect all presented 10-base sequences located in the barcode position and save a table with thier frequencies.

In [1]:
# forward reads, barcode 46
!zgrep 'CCAAGCGGTCTTAGGAAGACAA' ../data/test_fastq/F350009908_L01_46_1.fna.gz | awk -F 'CCAAGCGGTCTTAGGAAGACAA' '{print substr($2, 0, 10)}' | sort | uniq -c | sort -n > ../data/test_barcode_stats/46_stat_forward.txt

# reverse reads, barcode 46
!zgrep 'CGTTCTGTGAGCCAAGGAGTTG' ../data/test_fastq/F350009908_L01_46_2.fna.gz | awk -F 'CGTTCTGTGAGCCAAGGAGTTG' '{print substr($2, 0, 10)}' | sort | uniq -c | sort -n > ../data/test_barcode_stats/46_stat_reverse.txt

#### 2. Collect all MGI barcode sequences and their reverse variants

In [2]:
import pandas as pd
import numpy as np
from Bio.Seq import reverse_complement

import numpy as np

barcode_list = pd.read_csv(
    '../data/MGI_index_list.txt', 
    header = None,
    names = ('skip', 'Index', 'seq'),
    sep = ' '
)


barcode_list_rev = barcode_list.copy()
barcode_list_dict = {
    barcode_list.seq[i] : barcode_list.Index[i] for i in barcode_list.index
}


barcode_list_rev['seq'] = [reverse_complement(s) for s in barcode_list_rev['seq']]
barcode_list_rev_dict = {
    barcode_list_rev.seq[i] : barcode_list_rev.Index[i] for i in barcode_list_rev.index
}


#### 3. Estimate misdemultiplexing rate using the `zgrep` command results 

In [3]:
FORWARD = 'forward'
REVERSE = 'reverse'
barcode = '46'

folder = '../data/test_barcode_stats/'


rate = {}
total_bc = {}

for read_type in (FORWARD, REVERSE):

    data = pd.read_csv(
        f'{folder}/{barcode}_stat_{read_type}.txt', 
        sep=' ',
        skipinitialspace = True,
        header = None,
        names = ('Count', 'seq')
    )

    if len(data) == 0:
        continue


    if read_type is FORWARD:

        data = data[data.index.isin ([i for i in data.index if data.seq[i] in set(barcode_list.seq)]) ]
        data['Index'] = [barcode_list_dict[s] for s in data.seq]

        # extract the count of the target barcode
        for j in data.index:
            if data.Index[j] == int(barcode):
                target_count = data.Count[j]


    elif read_type is REVERSE:
        data = data[data.index.isin ([i for i in data.index if data.seq[i] in set(barcode_list_rev.seq)]) ]
        data['Index'] = [barcode_list_rev_dict[s] for s in data.seq]

        # extract the count of the target barcode
        for j in data.index:
            if data.Index[j] == int(barcode):
                target_count = data.Count[j]


    data.reset_index(drop=True, inplace=True)

    total = np.sum(data.Count)

    percent = (total - target_count) * 100/total

    rate[read_type] = percent
    total_bc[read_type] = total



print(
    f'Barcode: {barcode}', 
    f'Misdemultiplexing rate in forward reads: {round(rate[FORWARD], 2)}%', 
    f'Misdemultiplexing rate in reverse reads: {round(rate[REVERSE], 2)}%', 
    f'Number of forward reads observed: {total_bc[FORWARD]}', 
    f'Number of reverse reads observed: {total_bc[REVERSE]}', 
    
    sep='\n'
)


Barcode: 46
Misdemultiplexing rate in forward reads: 1.71%
Misdemultiplexing rate in reverse reads: 0.49%
Number of forward reads observed: 192786
Number of reverse reads observed: 60884


In [4]:
data

,Count,seq,Index
0,1,AATTACCATG,88
1,1,ACCAGCCTGA,80
2,2,AGACATGGTG,89
3,2,AGACCACGAT,77
4,2,CCAGACATAT,90
5,2,GTATTAGCCA,78
6,2,TTACAGTGCA,85
7,3,GCCAAGTCCA,45
8,3,TGTACACACC,94
9,4,CTCTGCACTG,79
